# Let's play with Andrej's code and get intuition

I'm not going do the exercises. It seems they are centered around tweaking the gradient step and finding better optimization techniques.

In [4]:
words = open('../external_resources/makemore/names.txt', 'r').read().splitlines()

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [6]:
C = torch.randn([27, 2])

In [7]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
X.shape

torch.Size([32, 3])

In [9]:
emb = C[X]

In [10]:
# 32 units, each unit is made of 3 numbers (context length) and the embedding size is 2
emb.shape

torch.Size([32, 3, 2])

In [11]:
# we want to apply a linear transformation to each unit which is is a 3x2 matrix (i.e. length 6)
W1 = torch.randn([6, 100])
b = torch.randn([100])

In [12]:
# this turns each 3x2 matrix into a single vector of length 6
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1)[3]

tensor([ 0.0973,  0.1952,  0.5919, -1.4099,  0.5919, -1.4099])

In [13]:
torch.cat(torch.unbind(emb, dim=1), dim=1)[3]

tensor([ 0.0973,  0.1952,  0.5919, -1.4099,  0.5919, -1.4099])

In [14]:
# fastest way to do the same thing
emb.view(32, 6)[3]
# same thing but guesses the first dimension
#emb.view(-1, 6)[3]

tensor([ 0.0973,  0.1952,  0.5919, -1.4099,  0.5919, -1.4099])

In [15]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b)

In [17]:
W2 = torch.randn([100, 27])
b2 = torch.randn([27])


In [25]:
logits = h @ W2 + b2
#counts = logits.exp()
#prob = counts / counts.sum(dim=1, keepdim=True)
#loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y)

tensor(14.1715)

In [27]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [41]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [1]:
27*10 + 30*200 + 200 + 200*27 + 27

11897

In [42]:
for p in parameters:
  p.requires_grad = True

In [43]:

lossi = []
stepi = []

In [50]:
def compute_loss(X, Y, C, W1, b1, W2, b2):
  emb = C[X]
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y)
  return loss.item()

In [52]:
batch_size = 32
for i in range(200000):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  
  # forward pass
  emb = C[Xtr[ix]] # (batch_size, 3, 10)
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # (batch_size, 200)
  logits = h @ W2 + b2 # (batch_size, 27)
  loss = F.cross_entropy(logits, Ytr[ix])

  # print every 1000 steps rewritting the same line
  if i % 1000 == 0:
    print(i, compute_loss(Xdev, Ydev, C, W1, b1, W2, b2), end='\r')
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

In [53]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carmahzadelle.
khy.
mili.
thiyah.
cassie.
rahnie.
amerynci.
aquil.
romara.
chaiir.
kaleigh.
ham.
pornie.
gates.
lilah.
jadiq.
watell.
dearynix.
kaellissa.
key.


My understanding:

This seems like the fundamental Neural Network Model before the advent of more fancier architectures. Seems important to understand.

A few observations:

* My stats intuition does not help me much here, this model is so different than any other stats model I've worked on. I realize I have to develop new intuition around Neural Networks.

* Roughly, this network architecture has an hourglass structure (1) finds a 10-dimensional embedding for each character (2) maps 3 points in 10 dimenions to 200 dimensions (3) applies an activation function in this 200 dimensional space (4) maps vector down to 27 dimensions. 

Intuively, it seems step #2 is linking the characters together (i.e. this is where most of the "thinking" happens) 

* I'm suprised how well this Neural Networks works. The names it generates sound quite real.

* This Neural Network has a lot of parameters: 27*10 + 30*200 + 200 + 200*27 + 27 ~ 1200 parameters (for about 200K data points).

* There is a lot of linear mappings here, in which some matrix "A" maps a vector x -> Ax. 

* The tanh function is... intruiging, sends (-inf, inf) -> (0, 1). I guess this introduces nonlinearity to this network and "contrains" the range. Is that necessary? What would happen without the tanh?


# - E01: Tune the hyperparameters of the training to beat my best validation loss of 2.2
